In [1]:
import sys
import torch
# custom
from AudioCaps.AudioCaps_Dataset import * # 데이터셋
from transformers import GPT2Tokenizer
from ClipCap_forAAC.CLIPCAP_forAAC import * # network
from Train import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

mapping_network_ver = 1
# PANNs를 써먹기 위해 prefix_size를 수정
audio_prefix_size = 15
semantic_prefix_size = 11 # 기존의 Semantic mapping network를 사용시
# semantic_prefix_size = 10 # 새로운 Semantic mapping network를 사용시
prefix_size = audio_prefix_size + semantic_prefix_size
transformer_num_layers = {"audio_num_layers" : 4 , "semantic_num_layers" : 4}
prefix_size_dict = {"audio_prefix_size" : audio_prefix_size, "semantic_prefix_size" : semantic_prefix_size}

USE_CUDA = torch.cuda.is_available() 
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

vocab_size = None
tokenizer_type = None

if vocab_size == None :
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer_type = 'GPT2'
else :
    tokenizer = tokenizer_AudioCaps(vocab_size)
    tokenizer_type = 'Custom'

model = get_ClipCap_AAC(tokenizer, mapping_network_ver = mapping_network_ver, 
                        vocab_size = vocab_size, Dataset = 'AudioCaps',
                        prefix_size_dict = prefix_size_dict, transformer_num_layers = transformer_num_layers, 
                        encoder_freeze = False, decoder_freeze = True,
                        pretrain_fromAudioCaps = False, device = device)

/home/cuai5th/anaconda3/envs/pytorch1.8_p38/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=512 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/cuai5th/anaconda3/envs/pytorch1.8_p38/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


audio feature's mapping network : num_head = 8 num_layers = 4
semantic feature ver1's mapping network : num_head = 8 num_layers = 4
GPT2 freezing


In [2]:
model_name = 'model_AudioCaps_' + tokenizer_type

if tokenizer_type == 'GPT2' :
    model_name += '.pt'
else :
    added_str = '_' + str(vocab_size) + '.pt'
    model_name += added_str

model.load_state_dict(torch.load("./Trained_model_params/" + model_name))

# save_folder_path = './ClipCap_forAAC/pre_trained_params_from_audiocaps/' + str(vocab_size) + '/'
# common_word = str(vocab_size) + '_in_Audiocaps.pt'

save_folder_path = './ClipCap_forAAC/pre_trained_params_from_audiocaps/GPT2/'
common_word = 'GPT2_in_Audiocaps.pt'

torch.save(model.audio_clip_project.state_dict(), save_folder_path + 'audio_clip_project_' + common_word)
torch.save(model.semantic_clip_project.state_dict(), save_folder_path + 'semantic_clip_project_' + common_word)
torch.save(model.audio_encoder.state_dict(), save_folder_path + 'audio_encoder_' + common_word)